In [42]:
import os
import json
import pandas as pd 
import traceback

In [43]:
from langchain.chat_models import ChatOpenAI

In [44]:
key=os.getenv("OPENAI_API_KEY")

In [45]:
from dotenv import load_dotenv
load_dotenv()

True

In [46]:
llm=ChatOpenAI(openai_api_key=key,model_name='gpt-3.5-turbo',temperature=0.6)

In [47]:
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain,SequentialChain

In [48]:
response_json={
    1:{"mcq":"question will be here",
     "options": {
            'a':"first option",
            'b':"second option",
            'c':"third option",
            'd':"fourth option"
        },
    
    "correct":"correct option"
    },
    2:{"mcq":"question will be here",
     "options": {
            'a':"first option",
            'b':"second option",
            'c':"third option",
            'd':"fourth option"
        },
    
    "correct":"correct option"
    }
}

In [49]:
template1="{text}.you have a text.now your task is to generate {number} mcq questions from this above text for {subject} student in {tone} tone.make sure that no mcq question is repeated and the questions are only from the above text.for generating mcq questions you have to follow the below response_json format.{response_json}"

In [50]:
quiz_generation_prompt1=PromptTemplate(
    input_variables=["text","number","subject","tone","response_json"],
    template=template1
)

In [51]:
quiz=LLMChain(llm=llm,prompt=quiz_generation_prompt1,output_key="quiz")

In [52]:
template2="You are an expert english grammarian and writer. Given a Multiple Choice Quiz for {subject} students.You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use at max 50 words for complexity analysis.if the quiz is not at per with the cognitive and analytical abilities of the students,update the quiz questions which needs to be changed and change the tone such that it perfectly fits the student abilities. Quiz_MCQs:{quiz}"

In [53]:
correct_answer_generation_prompt2=PromptTemplate(
    input_variables=["subject","quiz"],
    template=template2
)

In [54]:
answer=LLMChain(llm=llm,prompt=correct_answer_generation_prompt2,output_key="review")

In [55]:
mcq_chain=SequentialChain(chains=[quiz,answer],input_variables=["text","number","subject","tone","response_json"],output_variables=["quiz","review"])

In [56]:
path=r'C:\Users\USER\mcqgen\data.txt'
with open(path, 'r') as file:
    text=file.read()

In [57]:
response_json

{1: {'mcq': 'question will be here',
  'options': {'a': 'first option',
   'b': 'second option',
   'c': 'third option',
   'd': 'fourth option'},
  'correct': 'correct option'},
 2: {'mcq': 'question will be here',
  'options': {'a': 'first option',
   'b': 'second option',
   'c': 'third option',
   'd': 'fourth option'},
  'correct': 'correct option'}}

In [58]:
json.dumps(response_json)

'{"1": {"mcq": "question will be here", "options": {"a": "first option", "b": "second option", "c": "third option", "d": "fourth option"}, "correct": "correct option"}, "2": {"mcq": "question will be here", "options": {"a": "first option", "b": "second option", "c": "third option", "d": "fourth option"}, "correct": "correct option"}}'

In [59]:
mcq=mcq_chain({"text":text,"number":7,"subject":"Data Science","tone":"simple","response_json":json.dumps(response_json)})
quiz_table=mcq.get("quiz")
print(quiz_table)
quiz_table=json.loads(quiz_table)

{
"1": {
"mcq": "What type of data analysis examines data to gain insights into what happened or what is happening in the data environment?",
"options": {
"a": "Descriptive analysis",
"b": "Diagnostic analysis",
"c": "Predictive analysis",
"d": "Prescriptive analysis"
},
"correct": "a"
},
"2": {
"mcq": "Which type of data analysis involves a deep-dive or detailed examination to understand why something happened?",
"options": {
"a": "Descriptive analysis",
"b": "Diagnostic analysis",
"c": "Predictive analysis",
"d": "Prescriptive analysis"
},
"correct": "b"
},
"3": {
"mcq": "Which type of data analysis uses historical data to make accurate forecasts about data patterns that may occur in the future?",
"options": {
"a": "Descriptive analysis",
"b": "Diagnostic analysis",
"c": "Predictive analysis",
"d": "Prescriptive analysis"
},
"correct": "c"
},
"4": {
"mcq": "What type of data analysis suggests an optimum response to the predicted outcome?",
"options": {
"a": "Descriptive analysis",
"b

In [60]:
quiz_table

{'1': {'mcq': 'What type of data analysis examines data to gain insights into what happened or what is happening in the data environment?',
  'options': {'a': 'Descriptive analysis',
   'b': 'Diagnostic analysis',
   'c': 'Predictive analysis',
   'd': 'Prescriptive analysis'},
  'correct': 'a'},
 '2': {'mcq': 'Which type of data analysis involves a deep-dive or detailed examination to understand why something happened?',
  'options': {'a': 'Descriptive analysis',
   'b': 'Diagnostic analysis',
   'c': 'Predictive analysis',
   'd': 'Prescriptive analysis'},
  'correct': 'b'},
 '3': {'mcq': 'Which type of data analysis uses historical data to make accurate forecasts about data patterns that may occur in the future?',
  'options': {'a': 'Descriptive analysis',
   'b': 'Diagnostic analysis',
   'c': 'Predictive analysis',
   'd': 'Prescriptive analysis'},
  'correct': 'c'},
 '4': {'mcq': 'What type of data analysis suggests an optimum response to the predicted outcome?',
  'options': {'a

In [61]:
final_quiz_table=[]
for key,value in quiz_table.items():
    mcq=value['mcq']
    options=" | ".join([f"{option}: {option_value}" for option,option_value in value["options"].items()])
    correct=value["correct"]

    final_quiz_table.append({"MCQ":mcq,"Choices":options,"Correct Answer":correct})

In [62]:
df=pd.DataFrame(final_quiz_table)
df

,MCQ,Choices,Correct Answer
0,What type of data analysis examines data to ga...,a: Descriptive analysis | b: Diagnostic analys...,a
1,Which type of data analysis involves a deep-di...,a: Descriptive analysis | b: Diagnostic analys...,b
2,Which type of data analysis uses historical da...,a: Descriptive analysis | b: Diagnostic analys...,c
3,What type of data analysis suggests an optimum...,a: Descriptive analysis | b: Diagnostic analys...,d
4,Which technique is used in prescriptive analys...,a: Machine learning | b: Data mining | c: Fore...,a
5,What technique is used in predictive analysis ...,a: Data discovery | b: Pattern matching | c: M...,c
6,What type of analysis would a flight booking s...,a: Descriptive analysis | b: Diagnostic analys...,c


In [63]:
df.to_csv("Data_Science_Quiz.csv")